In [ ]:
from pathlib import Path
import re
from pathlib import Path
import pandas as pd

# Paths for your PC
LOG_DIR = Path("Path for logs")  # logs folder
RESULTS_DIR = Path("Path for results")  # results folder
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Logs folder:", LOG_DIR)
print("Files found:", [p.name for p in LOG_DIR.glob('*')])


In [ ]:
# Cell 2 — Verify files and preview
for fname in ['auth.log', 'system.log', 'web.log']:
    p = LOG_DIR / fname
    print("----", fname, "exists?", p.exists())
    if p.exists():
        print('\n'.join(p.read_text(errors='ignore').splitlines()[:6]))
    print()


In [ ]:
from pathlib import Path
import re
import pandas as pd

# Base folder for logs on your PC
DATA = Path(Base folder path)
auth_path = DATA / Path for auth.log


print("DATA folder:", DATA)
print("auth.log path:", auth_path)
print("auth.log exists?", auth_path.exists())



In [ ]:
if not auth_path.exists():
    raise FileNotFoundError(f"auth.log not found at {auth_path}. Put the file there or update DATA path.")

lines = auth_path.read_text(errors='ignore').splitlines()
print("Total lines:", len(lines))
print("First 8 lines:")
print("\n".join(lines[:8]))


In [ ]:
from pathlib import Path
import re
import pandas as pd

# Path to auth.log on your PC
auth_path = Path()

failed = []

# Read the log and find failed SSH attempts
with open(auth_path, 'r', errors='ignore') as f:
    for line in f:
        if ('Failed password' in line) or ('authentication failure' in line) or ('Failed publickey' in line):
            m = re.search(r'from\s+(\d{1,3}(?:\.\d{1,3}){3})', line)
            if m:
                parts = line.split()
                ts = " ".join(parts[0:3]) if len(parts) >= 3 else ""
                failed.append({'ts': ts, 'ip': m.group(1), 'raw': line.strip()})

# Convert to DataFrame
df_failed = pd.DataFrame(failed)

if df_failed.empty:
    print("No failed SSH entries found in auth.log")
else:
    # Count failed attempts per IP
    df_failed['count'] = df_failed.groupby('ip')['ip'].transform('count')
    df_failed = df_failed.sort_values(['count','ip'], ascending=[False, True]).reset_index(drop=True)
    display(df_failed.head(20))


In [ ]:
#summarise totals
print("Total failed logins:", len(df_failed))
print("Unique attacker IPs:", df_failed['ip'].nunique())
print("Top 5 attacker IPs:")
print(df_failed['ip'].value_counts().head())


In [ ]:
#coorelate with access.log 
web_path = Path()

if web_path.exists():
    web_lines = web_path.read_text(errors='ignore').splitlines()
    
    # Extract IPs from web.log
    ips_in_web = [
        re.search(r'(\d{1,3}(?:\.\d{1,3}){3})', l).group(1)
        for l in web_lines if re.search(r'(\d{1,3}(?:\.\d{1,3}){3})', l)
    ]
    
    # Find overlap
    overlap = set(df_failed['ip']).intersection(set(ips_in_web))
    print("IPs seen in BOTH auth.log and web.log:", overlap)
else:
    print("No web.log file found, skipping correlation.")


In [ ]:
report_path = Path()
report_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure the results folder exists

with open(report_path, "w") as f:
    f.write("CSDF Lab - Assignment 4 Findings\n")
    f.write("="*40 + "\n\n")
    f.write(f"Total failed logins: {len(df_failed)}\n")
    f.write(f"Unique attacker IPs: {df_failed['ip'].nunique()}\n\n")
    f.write("Top 5 attacker IPs:\n")
    f.write(str(df_failed['ip'].value_counts().head()))
    f.write("\n")

print("Report saved to:", report_path)

